<a href="https://colab.research.google.com/github/usamireko/NEUTRINO-Colab/blob/main/NEUTRINO_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Connect your Drive account to Colab (100% necessary because your Google Drive storage will save all the things downloaded)
import json
import urllib.request
import os
import glob
from google.colab import drive
drive.mount("/content/drive")

json_url = 'https://raw.githubusercontent.com/usamireko/NEUTRINO-Colab/refs/heads/main/download_info.json'

with urllib.request.urlopen(json_url) as response:
    config = json.load(response)

voice_data = config['voices']
neutrino = config['neutrino']

In [ ]:
import os
from IPython.display import clear_output

#@title Install/Update NEUTRINO
zip_path = f"/content/{neutrino['zip']}"


install_dir = "/content/drive/MyDrive/NEUTRINO"
!mkdir -p "{install_dir}"

print(f"Downloading NEUTRINO {neutrino['version']}...")
!gdown {neutrino['id']} -O "{zip_path}"

temp_extract_dir = "/content/neutrino_temp_extract"
!mkdir -p "{temp_extract_dir}"


!unzip "{zip_path}" -d "{temp_extract_dir}"

extracted_contents = os.listdir(temp_extract_dir)


extracted_folder_name = None
for item in extracted_contents:
    if os.path.isdir(os.path.join(temp_extract_dir, item)):
        extracted_folder_name = item
        break

if extracted_folder_name:
    source_folder = os.path.join(temp_extract_dir, extracted_folder_name)

    !rsync -av "{source_folder}/" "{install_dir}/"
    !rm -r "{temp_extract_dir}" #
    os.remove(zip_path)

    clear_output()
    print(f"NEUTRINO {neutrino['version']} installed at {install_dir}")
else:
    print(f"Error: Could not find the main extracted folder within {temp_extract_dir}")
    print(f"Contents of {temp_extract_dir}: {extracted_contents}")


In [ ]:
import os # Make sure os is imported
from IPython.display import clear_output
#@title Select a voicebank to download and install (Nakumo and Merrow are already downloaded)
voice = "Shikoku Metan"  #@param ["Kiritan", "Nakumo", "Tohoku Zunko", "Tohoku Itako", "Oedo Ochanko", "Shikoku Metan", "Runo", "Merrow", "Soma", "Zundamon", "Seven", "JSUT"]


vb = voice_data[voice]
zip_path = f"/content/{vb['zip']}"
temp_dir = "/content/vb_temp"
target_dir = f"/content/drive/MyDrive/NEUTRINO/model/{vb['folder']}"

print(f"Downloading {voice} voicebank...")

!gdown {vb['id']} -O "{zip_path}"

os.makedirs(temp_dir, exist_ok=True)

print("Extracting...")

if os.path.exists(zip_path):
    !unzip -j "{zip_path}" -d "{temp_dir}"
else:
    print(f"Error: Downloaded zip file not found at '{zip_path}'. Download might have failed or saved elsewhere.")
    exit()

print(f"Moving files to {target_dir}...")
os.makedirs(target_dir, exist_ok=True)
!rsync -av "{temp_dir}/" "{target_dir}/"
clear_output()
print("Cleaning up...")

if os.path.exists(zip_path):
    os.remove(zip_path)
!rm -rf "{temp_dir}"
print(f"{voice} voicebank installed or updated at {target_dir}")

In [ ]:
#@title Not sure about what VB´s you have? Click here!

!ls /content/drive/MyDrive/NEUTRINO/model

In [ ]:
#@title Download PAID voicebanks, not gonna provide installers
gdrive_ID = ID #@param
zip_name = a.zip #@param
singer_name = AKANE #@param
%cd /content
!gdown {gdrive_ID}
!mkdir /content/drive/MyDrive/NEUTRINO/model/{singer_name}
!unzip -j /content/{filename} /content/drive/MyDrive/NEUTRINO/model/{singer_name}
!rm /content/{filename}

# Time to run!

In [ ]:
#@title Run this first

%cd /content/drive/MyDrive/NEUTRINO

In [ ]:
#@title And then this
%%bash
find . -type f -exec chmod 755 \{\} \;


In [ ]:
#@title Upload the project file
from google.colab import files
uploaded = files.upload()

!mv *.musicxml /content/drive/MyDrive/NEUTRINO/score/musicxml






# For users that want a simple and easy run

In [ ]:
#@title Parameter config
BASENAME = "sample1" # @param {"type":"string"}
InferenceMode = "4" # @param ["2","3","4"]
ModelDir = "MERROW"   # @param ["KIRITAN","SEVEN","NAKUMO","YOKO","JSUT","ITAKO","ZUNKO","ZUNDAMON","METAN","MERROW","RUNO","OCHANKO","SOMA"] {"type":"raw"}
%cd /content/drive/MyDrive/NEUTRINO


In [ ]:
#@title  Render NSF and WORLD results
# Run All process

%%bash -s "$BASENAME" "$InferenceMode" "$ModelDir"
# Project settings
BASENAME=$1
NumThreads=4
InferenceMode=$2

# musicXML_to_label
SUFFIX=musicxml

# NEUTRINO
ModelDir=$3
StyleShift=0

# NSF
PitchShiftNsf=0

# WORLD
PitchShiftWorld=0
FormantShift=1.0
SmoothPitch=0.0
SmoothFormant=0.0
EnhanceBreathiness=0.0

if [ ${InferenceMode} -eq 4 ]; then
    NsfModel=va
    SamplingFreq=48
elif [ ${InferenceMode} -eq 3 ]; then
    NsfModel=vs
    SamplingFreq=48
elif [ ${InferenceMode} -eq 2 ]; then
    NsfModel=ve
    SamplingFreq=24
fi

# PATH to currennt library
export LD_LIBRARY_PATH=$PWD/bin:$PWD/NSF/bin:$LD_LIBRARY_PATH

echo "`date +"%M:%S.%2N"` : start MusicXMLtoLabel"
./bin/musicXMLtoLabel score/musicxml/${BASENAME}.${SUFFIX} score/label/full/${BASENAME}.lab score/label/mono/${BASENAME}.lab

echo "`date +"%M:%S.%2N"` : start NEUTRINO"
./bin/NEUTRINO score/label/full/${BASENAME}.lab score/label/timing/${BASENAME}.lab ./output/${BASENAME}.f0 ./output/${BASENAME}.melspec ./model/${ModelDir}/ -w ./output/${BASENAME}.mgc ./output/${BASENAME}.bap -n 1 -o ${NumThreads} -k ${StyleShift} -d ${InferenceMode} -g 0 -t

echo "`date +"%M:%S.%2N"` : start NSF"
./bin/NSF output/${BASENAME}.f0 output/${BASENAME}.melspec ./model/${ModelDir}/${NsfModel}.bin output/${BASENAME}.wav -l score/label/timing/${BASENAME}.lab -n 1 -p ${NumThreads} -s ${SamplingFreq} -f ${PitchShiftNsf} -g 0 -t

echo "`date +"%M:%S.%2N"` : start WORLD"
./bin/WORLD output/${BASENAME}.f0 output/${BASENAME}.mgc output/${BASENAME}.bap output/${BASENAME}_world.wav -f ${PitchShiftWorld} -m ${FormantShift} -p ${SmoothPitch} -c ${SmoothFormant} -b ${EnhanceBreathiness} -n ${NumThreads} -t

echo "`date +"%M:%S.%2N"` : END"

In [ ]:
#@title  Render NSF
# Run All process
%%bash -s "$BASENAME" "$InferenceMode" "$ModelDir"
# Project settings
BASENAME=$1
NumThreads=4
InferenceMode=$2

# musicXML_to_label
SUFFIX=musicxml

# NEUTRINO
ModelDir=$3
StyleShift=0

# NSF
PitchShiftNsf=0

# WORLD
PitchShiftWorld=0
FormantShift=1.0
SmoothPitch=0.0
SmoothFormant=0.0
EnhanceBreathiness=0.0

if [ ${InferenceMode} -eq 4 ]; then
    NsfModel=va
    SamplingFreq=48
elif [ ${InferenceMode} -eq 3 ]; then
    NsfModel=vs
    SamplingFreq=48
elif [ ${InferenceMode} -eq 2 ]; then
    NsfModel=ve
    SamplingFreq=24
fi

# PATH to currennt library
export LD_LIBRARY_PATH=$PWD/bin:$PWD/NSF/bin:$LD_LIBRARY_PATH

echo "`date +"%M:%S.%2N"` : start MusicXMLtoLabel"
./bin/musicXMLtoLabel score/musicxml/${BASENAME}.${SUFFIX} score/label/full/${BASENAME}.lab score/label/mono/${BASENAME}.lab

echo "`date +"%M:%S.%2N"` : start NEUTRINO"
./bin/NEUTRINO score/label/full/${BASENAME}.lab score/label/timing/${BASENAME}.lab ./output/${BASENAME}.f0 ./output/${BASENAME}.melspec ./model/${ModelDir}/ -w ./output/${BASENAME}.mgc ./output/${BASENAME}.bap -n 1 -o ${NumThreads} -k ${StyleShift} -d ${InferenceMode} -g 0 -t

echo "`date +"%M:%S.%2N"` : start NSF"
./bin/NSF output/${BASENAME}.f0 output/${BASENAME}.melspec ./model/${ModelDir}/${NsfModel}.bin output/${BASENAME}.wav -l score/label/timing/${BASENAME}.lab -n 1 -p ${NumThreads} -s ${SamplingFreq} -f ${PitchShiftNsf} -g 0 -t

echo "`date +"%M:%S.%2N"` : END"

In [ ]:
#@title Play the latest render
from IPython.display import Audio, display

if InferenceMode in ["3", "4"]:
    rate = 48000
else:
    rate = 24000

# Play the audio
display(Audio(f"/content/drive/MyDrive/NEUTRINO/output/{BASENAME}.wav", rate=rate, autoplay=True))


# For users that want more control of the output

In [ ]:
#@title Parameter config
BASENAME = "sample1" # @param {"type":"string"}
InferenceMode = "4"   # @param ["2", "3", "4"]
ModelDir = "KIRITAN" # @param ["KIRITAN","SEVEN","NAKUMO","YOKO","JSUT","ITAKO","ZUNKO","ZUNDAMON","METAN","MERROW","RUNO","OCHANKO","SOMA"]
StyleShift = 0 #@param {type:"slider", min:-1, max:1, step:0.1}
PitchShiftNsf = 0 #@param {type:"slider", min:-1, max:1, step:0.1}
PitchShiftWorld = 0 #@param {type:"slider", min:-1, max:1, step:0.1}
FormantShift = 1 # @param
SmoothPitch = 0.0 # @param
SmoothFormant = 0.0 # @param
EnhanceBreathiness = 0.0 # @param
NumThreads = 4
%cd /content/drive/MyDrive/NEUTRINO

In [ ]:

#@title Render NSF and WORLD results
# Run All process
%%bash -s "$BASENAME" "$InferenceMode" "$ModelDir" "$StyleShift" "$PitchShiftNsf" "$PitchShiftWorld" "$FormantShift" "$SmoothPitch" "$SmoothFormant" "$EnhanceBreathiness"
# Project settings
BASENAME=$1
NumThreads=4
InferenceMode=$2

# musicXML_to_label
SUFFIX=musicxml

# NEUTRINO
ModelDir=$3
StyleShift=$4

# NSF
PitchShiftNsf=$5

# WORLD
PitchShiftWorld=$6
FormantShift=$7
SmoothPitch=$8
SmoothFormant=$9
EnhanceBreathiness=$10

if [ ${InferenceMode} -eq 4 ]; then
    NsfModel=va
    SamplingFreq=48
elif [ ${InferenceMode} -eq 3 ]; then
    NsfModel=vs
    SamplingFreq=48
elif [ ${InferenceMode} -eq 2 ]; then
    NsfModel=ve
    SamplingFreq=24
fi

# PATH to currennt library
export LD_LIBRARY_PATH=$PWD/bin:$PWD/NSF/bin:$LD_LIBRARY_PATH

echo "`date +"%M:%S.%2N"` : start MusicXMLtoLabel"
./bin/musicXMLtoLabel score/musicxml/${BASENAME}.${SUFFIX} score/label/full/${BASENAME}.lab score/label/mono/${BASENAME}.lab

echo "`date +"%M:%S.%2N"` : start NEUTRINO"
./bin/NEUTRINO score/label/full/${BASENAME}.lab score/label/timing/${BASENAME}.lab ./output/${BASENAME}.f0 ./output/${BASENAME}.melspec ./model/${ModelDir}/ -w ./output/${BASENAME}.mgc ./output/${BASENAME}.bap -n 1 -o ${NumThreads} -k ${StyleShift} -d ${InferenceMode} -g 0 -t

echo "`date +"%M:%S.%2N"` : start NSF"
./bin/NSF output/${BASENAME}.f0 output/${BASENAME}.melspec ./model/${ModelDir}/${NsfModel}.bin output/${BASENAME}.wav -l score/label/timing/${BASENAME}.lab -n 1 -p ${NumThreads} -s ${SamplingFreq} -f ${PitchShiftNsf} -g 0 -t

echo "`date +"%M:%S.%2N"` : start WORLD"
./bin/WORLD output/${BASENAME}.f0 output/${BASENAME}.mgc output/${BASENAME}.bap output/${BASENAME}_world.wav -f ${PitchShiftWorld} -m ${FormantShift} -p ${SmoothPitch} -c ${SmoothFormant} -b ${EnhanceBreathiness} -n ${NumThreads} -t

echo "`date +"%M:%S.%2N"` : END"

In [ ]:
#@title Render NSF only
%%bash -s "$BASENAME" "$InferenceMode" "$ModelDir" "$StyleShift" "$PitchShiftNsf" "$PitchShiftWorld" "$FormantShift" "$SmoothPitch" "$SmoothFormant" "$EnhanceBreathiness"
# Project settings
BASENAME=$1
NumThreads=4
InferenceMode=$2

# musicXML_to_label
SUFFIX=musicxml

# NEUTRINO
ModelDir=$3
StyleShift=$4

# NSF
PitchShiftNsf=$5

# WORLD
PitchShiftWorld=$6
FormantShift=$7
SmoothPitch=$8
SmoothFormant=$9
EnhanceBreathiness=$10

if [ ${InferenceMode} -eq 4 ]; then
    NsfModel=va
    SamplingFreq=48
elif [ ${InferenceMode} -eq 3 ]; then
    NsfModel=vs
    SamplingFreq=48
elif [ ${InferenceMode} -eq 2 ]; then
    NsfModel=ve
    SamplingFreq=24
fi

# PATH to currennt library
export LD_LIBRARY_PATH=$PWD/bin:$PWD/NSF/bin:$LD_LIBRARY_PATH

echo "`date +"%M:%S.%2N"` : start MusicXMLtoLabel"
./bin/musicXMLtoLabel score/musicxml/${BASENAME}.${SUFFIX} score/label/full/${BASENAME}.lab score/label/mono/${BASENAME}.lab

echo "`date +"%M:%S.%2N"` : start NEUTRINO"
./bin/NEUTRINO score/label/full/${BASENAME}.lab score/label/timing/${BASENAME}.lab ./output/${BASENAME}.f0 ./output/${BASENAME}.melspec ./model/${ModelDir}/ -w ./output/${BASENAME}.mgc ./output/${BASENAME}.bap -n 1 -o ${NumThreads} -k ${StyleShift} -d ${InferenceMode} -g 0 -t

echo "`date +"%M:%S.%2N"` : start NSF"
./bin/NSF output/${BASENAME}.f0 output/${BASENAME}.melspec ./model/${ModelDir}/${NsfModel}.bin output/${BASENAME}.wav -l score/label/timing/${BASENAME}.lab -n 1 -p ${NumThreads} -s ${SamplingFreq} -f ${PitchShiftNsf} -g 0 -t

echo "`date +"%M:%S.%2N"` : start WORLD"
./bin/WORLD output/${BASENAME}.f0 output/${BASENAME}.mgc output/${BASENAME}.bap output/${BASENAME}_world.wav -f ${PitchShiftWorld} -m ${FormantShift} -p ${SmoothPitch} -c ${SmoothFormant} -b ${EnhanceBreathiness} -n ${NumThreads} -t

echo "`date +"%M:%S.%2N"` : END"

In [ ]:
#@title Play the latest render
from IPython.display import Audio, display

if InferenceMode in ["3", "4"]:
    rate = 48000
else:
    rate = 24000

# Play the audio
display(Audio(f"/content/drive/MyDrive/NEUTRINO/output/{BASENAME}.wav", rate=rate, autoplay=True))